In [9]:
# -*- coding: utf-8 -*-
"""modular_predict_abundancies_V1.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1Nis05JW3w5ActCq6HQKEyBE2DnKsyHgv
"""

# from google.colab import drive
# drive.mount('/content/drive')

import json

import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import xgboost as xgb
import pickle
import os

model_al_path = "../Desktop/Astral-Ray-Scratchpad-2/Soumik/ML/Trained_model/true_wt_al_model_final_new.pkl"
model_fe_path = "../Desktop/Astral-Ray-Scratchpad-2/Soumik/ML/Trained_model/true_wt_fe_model_final_new.pkl"
model_mg_path = "../Desktop/Astral-Ray-Scratchpad-2/Soumik/ML/Trained_model/true_wt_mg_model_final_new.pkl"
model_si_path = "../Desktop/Astral-Ray-Scratchpad-2/Soumik/ML/Trained_model/true_wt_si_model_final_new.pkl"

json_path = "../Desktop/ISRO.data_collection_v4.json"

# area_path='/content/drive/MyDrive/inter_IIT_ISRO/sampled-curve.txt'

# output_path_csv="/content/predictions.csv"

# def classify_lunar_feature(latitude, longitude):
#     # Primary classification: Highlands vs. Maria
#     if (40 <= latitude <= 90 or -90 <= latitude <= -40) and -180 <= longitude <= 180:
#         primary_class = "Highlands"
#     elif -20 <= latitude <= 40 and -60 <= longitude <= 60:
#         primary_class = "Maria"
#     else:
#         return "Highlands - General"

#     if primary_class == "Highlands":
#         if 40 <= latitude <= 60 and -40 <= longitude <= 10:
#             return "Highlands - Mountain Ranges"
#         elif 60 <= latitude <= 90 or -90 <= latitude <= -60:
#             return "Highlands - Craters"
#         elif 50 <= latitude <= 60 and -30 <= longitude <= -10:
#             return "Highlands - Valleys"
#         else:
#             return "Highlands - General"
#     elif primary_class == "Maria":
#         if -10 <= latitude <= 40 and -60 <= longitude <= -20:
#             return "Maria - Lunar Domes"
#         elif -10 <= latitude <= 30 and -40 <= longitude <= 10:
#             return "Maria - Rilles"
#         else:
#             return "Maria - Basaltic Plains"


def classify_lunar_feature(latitude: float, longitude: float):
    # Primary classification: Highlands vs. Maria
    if (40 <= latitude <= 90 or -90 <= latitude <= -40) and -180 <= longitude <= 180:
        primary_class = "Highlands"
    elif -20 <= latitude <= 40 and -60 <= longitude <= 60:
        primary_class = "Maria"
    else:
        return "Highlands - General"

    if primary_class == "Highlands":
        if 40 <= latitude <= 60 and -40 <= longitude <= 10:
            return "Highlands - Mountain Ranges"
        elif 60 <= latitude <= 90 or -90 <= latitude <= -60:
            return "Highlands - Craters"
        elif 50 <= latitude <= 60 and -30 <= longitude <= -10:
            return "Highlands - Valleys"
        else:
            return "Highlands - General"
    elif primary_class == "Maria":
        if -10 <= latitude <= 40 and -60 <= longitude <= -20:
            return "Maria - Lunar Domes"
        elif -10 <= latitude <= 30 and -40 <= longitude <= 10:
            return "Maria - Rilles"
        else:
            return "Maria - Basaltic Plains"


def red_chi_2(element: str, df: pd.DataFrame):
    df[f"red_chi_2_{element}"] = df[f"chi_2_{element}"] / (df[f"dof_{element}"])
    return df


def expand_dict_column(df, column_name, prefix):
    """
    Expand a dictionary column in a DataFrame into individual columns with a specified prefix.

    Parameters:
        df (pd.DataFrame): The input DataFrame.
        column_name (str): The name of the column containing dictionaries.
        prefix (str): The prefix to use for the new columns.

    Returns:
        pd.DataFrame: The DataFrame with new columns added.
    """
    # Ensure the column exists and contains dictionaries
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in the DataFrame.")

    # Check if the first non-null element is a dictionary
    first_non_null = df[column_name].dropna().iloc[0]
    if not isinstance(first_non_null, dict):
        raise ValueError(f"Column '{column_name}' does not contain dictionaries.")

    # Loop over the keys of the first dictionary
    for key in first_non_null.keys():
        new_column_name = f"{prefix}{key}"
        df[new_column_name] = df[column_name].apply(lambda x: x.get(key) if pd.notnull(x) else None)

    return df


model_paths = {
    "model_al": model_al_path,
    "model_fe": model_fe_path,
    "model_mg": model_mg_path,
    "model_si": model_si_path,
}

# Define the feature sets for each model
features = {
    "model_al": [
        "wt_al",
        "wt_si",
        "photon_counts",
        "solar_zenith_angle",
        "emission_angle",
        "altitude",
        "exposure",
        "peak_si_c",
        "peak_mg_c",
        "peak_al_c",
        "peak_ti_c",
        "peak_fe_c",
        "peak_ca_c",
        "latitude",
        "longitude",
        "Region_Highlands - Craters",
        "Region_Highlands - Mountain Ranges",
        "Region_Maria - Basaltic Plains",
        "Region_Maria - Lunar Domes",
        "Region_Maria - Rilles",
    ],
    "model_fe": [
        "wt_mg",
        "wt_al",
        "wt_si",
        "wt_fe",
        "photon_counts",
        "solar_zenith_angle",
        "emission_angle",
        "altitude",
        "exposure",
        "peak_si_c",
        "peak_mg_c",
        "peak_al_c",
        "peak_ti_c",
        "peak_fe_c",
        "peak_ca_c",
        "latitude",
        "longitude",
        "Region_Highlands - Craters",
        "Region_Highlands - Mountain Ranges",
        "Region_Maria - Basaltic Plains",
        "Region_Maria - Lunar Domes",
        "Region_Maria - Rilles",
    ],
    "model_mg": [
        "wt_mg",
        "wt_al",
        "wt_si",
        "photon_counts",
        "solar_zenith_angle",
        "emission_angle",
        "altitude",
        "exposure",
        "peak_si_c",
        "peak_mg_c",
        "peak_al_c",
        "peak_ti_c",
        "peak_fe_c",
        "peak_ca_c",
        "latitude",
        "longitude",
        "Region_Highlands - Craters",
        "Region_Highlands - Mountain Ranges",
        "Region_Maria - Basaltic Plains",
        "Region_Maria - Lunar Domes",
        "Region_Maria - Rilles",
    ],
    "model_si": [
        "wt_si",
        "photon_counts",
        "solar_zenith_angle",
        "emission_angle",
        "altitude",
        "exposure",
        "peak_si_c",
        "peak_mg_c",
        "peak_al_c",
        "peak_ti_c",
        "peak_fe_c",
        "peak_ca_c",
        "latitude",
        "longitude",
        "Region_Highlands - Craters",
        "Region_Highlands - Mountain Ranges",
        "Region_Maria - Basaltic Plains",
        "Region_Maria - Lunar Domes",
        "Region_Maria - Rilles",
        "Region_Highlands - Valleys",
    ],
}


def one_hot_encode_region(df, column_name):
    """
    One-hot encodes the specified column in a DataFrame for the given regions.
    If the column is not present, raises an error.
    If the column contains values not in the predefined list, ignores them.

    Args:
        df (pd.DataFrame): The input DataFrame.
        column_name (str): The column to one-hot encode.

    Returns:
        pd.DataFrame: DataFrame with one-hot encoded columns added.
    """
    # Predefined categories for one-hot encoding
    allowed_categories = [
        "Highlands - Craters",
        "Highlands - Mountain Ranges",
        "Maria - Basaltic Plains",
        "Maria - Lunar Domes",
        "Maria - Rilles",
        "Highlands - Valleys",
    ]

    # Check if the column exists in the DataFrame
    if column_name not in df.columns:
        raise ValueError(f"The column '{column_name}' does not exist in the DataFrame.")

    # One-hot encoding logic
    for category in allowed_categories:
        encoded_column = f"Region_{category}"
        df[encoded_column] = (df[column_name] == category).astype(int)

    return df


def abun_pred(json_path, model_paths, features):
    df = pd.read_json(json_path)

    #   df=df.iloc[:10]

    df = expand_dict_column(df, "wt", "wt_")
    df = expand_dict_column(df, "dof", "dof_")
    df = expand_dict_column(df, "chi_2", "chi_2_")
    df = df = expand_dict_column(df, "computed_metadata", "")
    # print(df.columns)
    df["Region"] = df.apply(lambda row: classify_lunar_feature(row["latitude"], row["longitude"]), axis=1)

    df = red_chi_2("mg", df)
    df = red_chi_2("al", df)
    df = red_chi_2("si", df)
    df = red_chi_2("fe", df)
    df = red_chi_2("ti", df)
    df = red_chi_2("ca", df)

    data = one_hot_encode_region(df, "Region")

    data["emission_angle"] = 90 - data["emission_angle"]
    data["solar_zenith_angle"] = 90 - data["solar_zenith_angle"]

    data["lat"] = data["latitude"]
    data["long"] = data["longitude"]
    # data=data.drop(columns=['Region','Region_Highlands - General'])
    # print(data.columns)
    # print(data.head())
    predictions = {}

    for model_name, model_path in model_paths.items():
        # Check if the model file exists
        if not os.path.exists(model_path):
            raise FileNotFoundError(f"Model file for {model_name} not found at path: {model_path}")

        # Load the model using pickle
        with open(model_path, "rb") as file:
            model = pickle.load(file)

        # Retrieve the feature set for the current model
        feature_set = features[model_name]

        # Check if all required features are present in the data
        missing_features = [feature for feature in feature_set if feature not in data.columns]
        if missing_features:
            raise ValueError(f"Missing features for {model_name}: {missing_features}")

        # Prepare the input data for prediction
        model_input = data[feature_set]

        # Make predictions
        preds = model.predict(model_input)

        # Store the predictions with a descriptive column name
        prediction_column = f"{model_name}_prediction"
        predictions[prediction_column] = preds

    predictions_df = pd.DataFrame(predictions)
    combined_df = pd.concat([data[["latitude", "longitude", "wt_fe", "wt_al", "wt_mg", "wt_si"]], predictions_df], axis=1)
    # print(combined_df.columns)
    combined_df_json = combined_df.to_json(orient="records")
    return combined_df_json
    # combined_df.to_csv(output_path)


print(abun_pred(json_path, model_paths, features))


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [11]:
import numpy as np

# Generate an array of 10 points between 1 and 10
arr = np.linspace(1, 10, num=8)
print(arr)


[ 1.          2.28571429  3.57142857  4.85714286  6.14285714  7.42857143
  8.71428571 10.        ]
